# Limpieza y unión de datos

## Instalación de librerías

In [1]:
import pandas as pd
import geopandas as gpd

## Carga de datos

In [2]:
datos = pd.read_csv(r'C:\Users\EMMY\Desktop\Proyecto\Limpieza\datos.csv')

## Agregar subregiones

In [3]:
# Subregiones
subregiones_shapefile = r'C:\Users\EMMY\Desktop\Proyecto\DB Original\Subregiones_-_Provincias_de_Colombia\Subregiones_-_Provincias_de_Colombia.shp'
sub = gpd.read_file(subregiones_shapefile)

# Municipios
municipios_shapefile = r'C:\Users\EMMY\Desktop\Proyecto\DB Original\MGN2023_MPIO_POLITICO\MGN_ADM_MPIO_GRAFICO.shp'
mun = gpd.read_file(municipios_shapefile)

# Ver las columnas de los archivos
print(sub.columns)
print(mun.columns)

Index(['OBJECTID', 'COD_SUBREG', 'NOM_SUBREG', 'COD_DEPTO', 'ShapeSTAre',
       'ShapeSTLen', 'geometry'],
      dtype='object')
Index(['dpto_ccdgo', 'mpio_ccdgo', 'mpio_cdpmp', 'dpto_cnmbr', 'mpio_cnmbr',
       'mpio_crslc', 'mpio_tipo', 'mpio_narea', 'mpio_nano', 'shape_Leng',
       'shape_Area', 'geometry'],
      dtype='object')


In [4]:
mun.head(1)

,dpto_ccdgo,mpio_ccdgo,mpio_cdpmp,dpto_cnmbr,mpio_cnmbr,mpio_crslc,mpio_tipo,mpio_narea,mpio_nano,shape_Leng,shape_Area,geometry
0,05,001,05001,ANTIOQUIA,MEDELLÍN,1965,MUNICIPIO,374.834005,2023,1.03538,0.030608,"POLYGON ((-75.66974 6.3736, -75.66965 6.3736, ..."


### Eliminar columnas innecesarias

In [5]:
sub = sub.drop(columns= ['OBJECTID', 'COD_SUBREG', 'ShapeSTAre', 'ShapeSTLen'])

In [6]:
mun = mun.drop(columns= ['mpio_ccdgo', 'mpio_crslc', 'mpio_tipo', 'mpio_narea', 'mpio_nano', 'shape_Leng', 'shape_Area'])

### Extraer los registros de Bolívar y Córdoba

In [7]:
departamento = ['13', '23']
departamento

['13', '23']

In [8]:
# Filtrar los archivos shp para los registros de Córdoba y Bolívar

sub1 = sub[sub['COD_DEPTO'].isin(departamento)].reset_index() #Departamentos
mun1 = mun[mun['dpto_ccdgo'].isin(departamento)].reset_index() #Municipios

In [ ]:
print(f'existen {datos['codigo_dane_municipio'].nunique()} municipios en el archivo datos')

existen 76 municipios en el archivo datos


In [10]:
print(f'sub1 tiene {sub1.shape[0]} registros y {sub1['NOM_SUBREG'].nunique()} subregiones diferentes')

sub1 tiene 13 registros y 13 subregiones diferentes


In [11]:
print(f'mun1 tiene {mun1.shape[0]} registros y {mun1['mpio_cnmbr'].nunique()} municipios diferentes')

mun1 tiene 76 registros y 76 municipios diferentes


### Unir los 2 archivos shp

In [12]:
# Asegurarse de que ambos GeoDataFrames tengan el mismo sistema de coordenadas
sub1 = sub1.to_crs(mun1.crs)

In [13]:
# Realizar una unión espacial para agregar la subregión a cada municipio
mun2 =  gpd.sjoin(mun1, sub1[['NOM_SUBREG', 'geometry']], how='left', predicate='intersects')

In [14]:
# Verificamos que no existen valores nulos
mun2.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 158 entries, 0 to 75
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   index        158 non-null    int64   
 1   dpto_ccdgo   158 non-null    object  
 2   mpio_cdpmp   158 non-null    object  
 3   dpto_cnmbr   158 non-null    object  
 4   mpio_cnmbr   158 non-null    object  
 5   geometry     158 non-null    geometry
 6   index_right  158 non-null    int64   
 7   NOM_SUBREG   158 non-null    object  
dtypes: geometry(1), int64(2), object(5)
memory usage: 11.1+ KB


Al inspeccionar el DataFrame mun2 con el método .info(), se observa que el número de registros aumentó de 76 a 158. Esto ocurre porque, según las coordenadas de latitud y longitud, algunos municipios están asociados a más de una subregión. Para optimizar el análisis, se seleccionará la subregión que ocupe la mayor proporción del área en cada municipio.

In [15]:
# Asegurarte de reproyectar tus datos a un CRS proyectado
mun1 = mun1.to_crs('EPSG:3116')
sub1 = sub1.to_crs('EPSG:3116')

# Realizar la unión espacial con todas las subregiones
mun3 = gpd.sjoin(mun1, sub1[['NOM_SUBREG', 'geometry']], how='left', predicate='intersects')

In [16]:
mun3 = mun3.drop(columns=['index', 'index_right'])

In [17]:
# Calcular el área de intersección para los municipios que tienen intersección
mun3['area_intersection'] = mun3.apply(
    lambda row: row.geometry.intersection(sub1.loc[sub1['NOM_SUBREG'] == row.NOM_SUBREG, 'geometry'].values[0]).area
    if pd.notna(row.NOM_SUBREG) else 0, axis=1
)

In [18]:
mun3 = mun3.reset_index()
mun3.columns

Index(['index', 'dpto_ccdgo', 'mpio_cdpmp', 'dpto_cnmbr', 'mpio_cnmbr',
       'geometry', 'NOM_SUBREG', 'area_intersection'],
      dtype='object')

In [19]:
mun3 = mun3.drop(columns= 'index')
mun3.columns

Index(['dpto_ccdgo', 'mpio_cdpmp', 'dpto_cnmbr', 'mpio_cnmbr', 'geometry',
       'NOM_SUBREG', 'area_intersection'],
      dtype='object')

In [20]:
# Seleccionar el registro con el área de intersección más grande por municipio
mun3_max_area = mun3.loc[mun3.groupby('mpio_cdpmp')['area_intersection'].idxmax()]
mun3_max_area

,dpto_ccdgo,mpio_cdpmp,dpto_cnmbr,mpio_cnmbr,geometry,NOM_SUBREG,area_intersection
0,13,13001,BOLÍVAR,CARTAGENA DE INDIAS,"MULTIPOLYGON (((769766.263 1530601.606, 769799...",DIQUE BOLIVARENSE,5.666435e+08
1,13,13006,BOLÍVAR,ACHÍ,"POLYGON ((949670.931 1469163.561, 949682.178 1...",MOJANA BOLIVARENSE,9.489217e+08
3,13,13030,BOLÍVAR,ALTOS DEL ROSARIO,"POLYGON ((996092.313 1469127.072, 996099.736 1...",LOBA,3.031439e+08
4,13,13042,BOLÍVAR,ARENAL,"POLYGON ((1021697.028 1428592.627, 1021697.017...",MAGDALENA MEDIO BOLIVARENSE,4.602459e+08
8,13,13052,BOLÍVAR,ARJONA,"POLYGON ((866038.263 1635342.895, 866063.189 1...",DIQUE BOLIVARENSE,5.871317e+08
...,...,...,...,...,...,...,...
146,23,23682,CÓRDOBA,SAN JOSÉ DE URÉ,"POLYGON ((840485.581 1370652.785, 840590.527 1...",SAN JORGE,5.254161e+08
148,23,23686,CÓRDOBA,SAN PELAYO,"POLYGON ((796652.476 1495620.852, 796655.896 1...",SINÚ MEDIO,4.398211e+08
151,23,23807,CÓRDOBA,TIERRALTA,"POLYGON ((791532.867 1413292.089, 792521.486 1...",ALTO SINÚ,4.915535e+09
154,23,23815,CÓRDOBA,TUCHÍN,"POLYGON ((843961.394 1518775.038, 843980.196 1...",SABANAS,1.026547e+08


In [21]:
# Resetear el índice
mun3 = mun3_max_area.reset_index(drop=True)

In [22]:
# Si un municipio no tiene intersección, sus columnas de subregión quedan como NaN
mun3 = mun3[['mpio_cdpmp', 'mpio_cnmbr', 'NOM_SUBREG']]

In [23]:
mun3.shape

(76, 3)

In [24]:
mun3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   mpio_cdpmp  76 non-null     object
 1   mpio_cnmbr  76 non-null     object
 2   NOM_SUBREG  76 non-null     object
dtypes: object(3)
memory usage: 1.9+ KB


In [25]:
mun3['mpio_cdpmp'].nunique()

76

In [26]:
mun3['NOM_SUBREG'].nunique()

13

In [27]:
mun3.columns

Index(['mpio_cdpmp', 'mpio_cnmbr', 'NOM_SUBREG'], dtype='object')

In [28]:
datos.columns

Index(['codigo_dane_departamento', 'departamento', 'codigo_dane_municipio',
       'municipio', 'grupo_cultivo', 'subgrupo', 'cultivo',
       'desagregacion_cultivo', 'año', 'periodo', 'area_sembrada',
       'area_cosechada', 'produccion', 'rendimiento', 'ciclo_cultivo'],
      dtype='object')

In [29]:
mun3.columns = ['codigo_dane_municipio', 'mpio_cnmbr', 'NOM_SUBREG']
mun3

,codigo_dane_municipio,mpio_cnmbr,NOM_SUBREG
0,13001,CARTAGENA DE INDIAS,DIQUE BOLIVARENSE
1,13006,ACHÍ,MOJANA BOLIVARENSE
2,13030,ALTOS DEL ROSARIO,LOBA
3,13042,ARENAL,MAGDALENA MEDIO BOLIVARENSE
4,13052,ARJONA,DIQUE BOLIVARENSE
...,...,...,...
71,23682,SAN JOSÉ DE URÉ,SAN JORGE
72,23686,SAN PELAYO,SINÚ MEDIO
73,23807,TIERRALTA,ALTO SINÚ
74,23815,TUCHÍN,SABANAS


In [30]:
mun3 = mun3.sort_values('mpio_cnmbr').reset_index()

In [31]:
mun3.dtypes

index                     int64
codigo_dane_municipio    object
mpio_cnmbr               object
NOM_SUBREG               object
dtype: object

In [32]:
mun3 = mun3.drop(columns='index')
mun3.dtypes

codigo_dane_municipio    object
mpio_cnmbr               object
NOM_SUBREG               object
dtype: object

### Unir subregiones a data

In [33]:
datos['codigo_dane_municipio'] = datos['codigo_dane_municipio'].astype('object')

In [34]:
datos['codigo_dane_municipio'].unique()

array([13006, 13030, 13042, 13052, 13062, 13074, 13140, 13160, 13001,
       13188, 13222, 13212, 13244, 13248, 13268, 13300, 13430, 13433,
       13440, 13442, 13468, 13458, 13473, 13490, 13549, 13580, 13600,
       13620, 13647, 13650, 13654, 13655, 13657, 13667, 13670, 13673,
       13683, 13688, 13744, 13760, 13780, 13810, 13836, 13838, 13873,
       13894, 23068, 23079, 23090, 23162, 23168, 23182, 23189, 23300,
       23350, 23417, 23419, 23464, 23466, 23001, 23500, 23555, 23570,
       23574, 23580, 23586, 23660, 23670, 23672, 23675, 23678, 23682,
       23686, 23807, 23815, 23855], dtype=object)

In [35]:
mun3['codigo_dane_municipio'].unique()

array(['13006', '13030', '13042', '13052', '13062', '23068', '13074',
       '23079', '13140', '23090', '13160', '13001', '23162', '23168',
       '23182', '13188', '23189', '13222', '23300', '13212', '13244',
       '13248', '13268', '13300', '23350', '23417', '23419', '13430',
       '13433', '13440', '13442', '23464', '13458', '23466', '23001',
       '13473', '23500', '13490', '13549', '23555', '23570', '23574',
       '23580', '23586', '13580', '13600', '23660', '23670', '23672',
       '23675', '23678', '13620', '13647', '13650', '13654', '13655',
       '23682', '13657', '13667', '13670', '23686', '13673', '13468',
       '13683', '13688', '13744', '13760', '13780', '23807', '13810',
       '23815', '13836', '13838', '23855', '13873', '13894'], dtype=object)

La diferencia principal entre los valores únicos de codigo_dane_municipio en data y mun3 es que en data los valores son de tipo entero (int), mientras que en mun3 los valores son de tipo cadena (str). Esto puede causar problemas al realizar operaciones de comparación o uniones entre estos conjuntos de datos.

In [36]:
# Convertir codigo_dane_municipio en mun3 a tipo int y luego a object
mun3['codigo_dane_municipio'] = mun3['codigo_dane_municipio'].astype(int)
mun3['codigo_dane_municipio'] = mun3['codigo_dane_municipio'].astype('object')

In [37]:
mun3['codigo_dane_municipio'].unique()

array([13006, 13030, 13042, 13052, 13062, 23068, 13074, 23079, 13140,
       23090, 13160, 13001, 23162, 23168, 23182, 13188, 23189, 13222,
       23300, 13212, 13244, 13248, 13268, 13300, 23350, 23417, 23419,
       13430, 13433, 13440, 13442, 23464, 13458, 23466, 23001, 13473,
       23500, 13490, 13549, 23555, 23570, 23574, 23580, 23586, 13580,
       13600, 23660, 23670, 23672, 23675, 23678, 13620, 13647, 13650,
       13654, 13655, 23682, 13657, 13667, 13670, 23686, 13673, 13468,
       13683, 13688, 13744, 13760, 13780, 23807, 13810, 23815, 13836,
       13838, 23855, 13873, 13894], dtype=object)

In [38]:
# Realizamos el merge
data_merged = pd.merge(datos, mun3[['codigo_dane_municipio', 'NOM_SUBREG']], 
                       on='codigo_dane_municipio', how='left')

In [39]:
# Si hay repeticiones y no es esperado, podemos detectar conflictos:
conflictos = mun3.groupby('codigo_dane_municipio')['NOM_SUBREG'].nunique()
conflictos = conflictos[conflictos > 1]

if not conflictos.empty:
    print("Se detectaron municipios asociados a múltiples subregiones:")
    print(conflictos)
else:
    print("Asignación de subregiones completada sin conflictos.")

Asignación de subregiones completada sin conflictos.


In [40]:
datos.shape

(15727, 15)

In [41]:
data_merged.shape

(15727, 16)

In [42]:
data_merged

,codigo_dane_departamento,departamento,codigo_dane_municipio,municipio,grupo_cultivo,subgrupo,cultivo,desagregacion_cultivo,año,periodo,area_sembrada,area_cosechada,produccion,rendimiento,ciclo_cultivo,NOM_SUBREG
0,13,Bolívar,13006,Achí,Frutales,Aguacate,Aguacate,Aguacate,2007,2007,80.0,80.0,800.0,10.0,Permanente,MOJANA BOLIVARENSE
1,13,Bolívar,13006,Achí,Frutales,Aguacate,Aguacate,Aguacate,2008,2008,80.0,80.0,720.0,9.0,Permanente,MOJANA BOLIVARENSE
2,13,Bolívar,13006,Achí,Frutales,Aguacate,Aguacate,Aguacate,2009,2009,80.0,80.0,800.0,10.0,Permanente,MOJANA BOLIVARENSE
3,13,Bolívar,13006,Achí,Frutales,Aguacate,Aguacate,Aguacate,2010,2010,80.0,80.0,480.0,6.0,Permanente,MOJANA BOLIVARENSE
4,13,Bolívar,13006,Achí,Frutales,Aguacate,Aguacate,Aguacate,2011,2011,80.0,80.0,560.0,7.0,Permanente,MOJANA BOLIVARENSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15722,23,Córdoba,23855,Valencia,Raíces Y Tubérculos,Raíces Y Tubérculos,Yuca,Yuca Consumo En Fresco,2021,2021A,75.0,72.0,864.0,12.0,Transitorio,ALTO SINÚ
15723,23,Córdoba,23855,Valencia,Raíces Y Tubérculos,Raíces Y Tubérculos,Yuca,Yuca Consumo En Fresco,2021,2021B,75.0,72.0,792.0,11.0,Transitorio,ALTO SINÚ
15724,23,Córdoba,23855,Valencia,Raíces Y Tubérculos,Raíces Y Tubérculos,Yuca,Yuca Consumo En Fresco,2022,2022A,85.0,65.0,780.0,12.0,Transitorio,ALTO SINÚ
15725,23,Córdoba,23855,Valencia,Raíces Y Tubérculos,Raíces Y Tubérculos,Yuca,Yuca Consumo En Fresco,2023,2023A,100.0,85.0,1020.0,12.0,Transitorio,ALTO SINÚ


In [43]:
data_merged = data_merged.map(lambda row: row.title() if isinstance(row, str) else row)
data_merged.head(2)

,codigo_dane_departamento,departamento,codigo_dane_municipio,municipio,grupo_cultivo,subgrupo,cultivo,desagregacion_cultivo,año,periodo,area_sembrada,area_cosechada,produccion,rendimiento,ciclo_cultivo,NOM_SUBREG
0,13,Bolívar,13006,Achí,Frutales,Aguacate,Aguacate,Aguacate,2007,2007,80.0,80.0,800.0,10.0,Permanente,Mojana Bolivarense
1,13,Bolívar,13006,Achí,Frutales,Aguacate,Aguacate,Aguacate,2008,2008,80.0,80.0,720.0,9.0,Permanente,Mojana Bolivarense


In [45]:
data_merged.columns = ['codigo_dane_departamento', 'departamento', 'codigo_dane_municipio',
       'municipio', 'grupo_cultivo', 'subgrupo', 'cultivo',
       'desagregacion_cultivo', 'año', 'periodo', 'area_sembrada',
       'area_cosechada', 'produccion', 'rendimiento', 'ciclo_cultivo',
       'subregion']

## Guardar df en un nuevo archivo

In [46]:
data_merged.to_csv('data.csv', index=False)